In [ ]:
import mne
import numpy as np

In [ ]:
import pandas as pd
participants = pd.read_csv('Dataset/participants.tsv', sep='\t')
print(participants.head(5))  # To inspect the first few rows

In [ ]:
import os

# List of subject IDs (assuming you have filenames like sub-pd01, sub-hc01, etc.)
subjects = participants.participant_id # List all subjects here
subjects=subjects.to_list()
sessions = ['ses-off', 'ses-on']  # for Parkinson's patients, if available

print(subjects)

In [ ]:
import os

# List of subject IDs (assuming you have filenames like sub-pd01, sub-hc01, etc.)
subjects = participants.participant_id # List all subjects here
subjects=subjects.to_list()
sessions = ['ses-off', 'ses-on']  # for Parkinson's patients, if available

print(subjects)

In [ ]:
subject_files_pd_on=[]
subject_files_pd_off=[]
subject_files_hc=[]

for subject in subjects:
    if 'pd' in subject:
        for session in sessions:
            file_path = f"Dataset/{subject}/{session}/eeg/{subject}_{session}_task-rest_eeg.bdf"
            if session == 'ses-on':
                subject_files_pd_on.append(file_path)
            else:
                subject_files_pd_off.append(file_path)
    elif 'hc' in subject:
        session = 'ses-hc'
        file_path = f"Dataset/{subject}/{session}/eeg/{subject}_{session}_task-rest_eeg.bdf"
        subject_files_hc.append(file_path)

print(subject_files_pd_on)
print(subject_files_pd_off)
print(subject_files_hc)

In [ ]:
def set_montage(raw_data):
    montage = mne.channels.make_standard_montage('biosemi32')
    raw_data.set_montage(montage, on_missing='warn')
    return raw_data

In [ ]:
def bandpass_filter(raw_data, l_freq=0.5, h_freq=50.0):
    raw_data.filter(l_freq=l_freq, h_freq=h_freq)
    return raw_data

In [ ]:
# Use T7 or T8 as proxy ECG channels (experimental approach)
def find_ecg_via_temporal_channels(ica, raw_data):
    # Experimentally identify ECG-like artifacts using temporal channels
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw_data, ch_name='T7')
    ica.exclude += ecg_indices  # Exclude identified ECG-like components
    return ica

In [ ]:
from mne.preprocessing import ICA
def apply_ica(raw_data, n_components=32):
    ica = ICA(n_components=n_components, random_state=97, max_iter="auto")
    ica.fit(raw_data)
    
    # Detect artifacts
    eog_indices, _ = ica.find_bads_eog(raw_data,ch_name=['Fp2', 'F8'],threshold=1.96)  # Detect eye blink components
    
    # Mark components for removal
    ica.exclude = eog_indices
    # Experimental ECG detection
    ica = find_ecg_via_temporal_channels(ica, raw_data)
    
    # Apply ICA to remove artifacts
    raw_data = ica.apply(raw_data)
    return raw_data

In [ ]:
def segment_data(raw_data, duration=1.0):
    events = mne.make_fixed_length_events(raw_data, duration=duration)
    epochs = mne.Epochs(raw_data, events, tmin=0, tmax=duration, baseline=None, preload=True)
    eeg_data = epochs.get_data()  # Shape should be (180, 32, 512) if 3 mins, 32 channels, 512 samples/s
    return eeg_data

In [ ]:
# Define frequency bands
freq_bands = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (13, 30),
    "gamma": (30, 48)
}

In [ ]:

def compute_psd(eeg_data, sfreq):
    psd_features = {}
    for band, (low, high) in freq_bands.items():
        psd_band, _ = mne.time_frequency.psd_array_multitaper(
            eeg_data, sfreq=sfreq, fmin=low, fmax=high, adaptive=True, normalization='full'
        )
        psd_features[band] = psd_band.mean(axis=2)  # Average PSD across time
    return psd_features

In [ ]:
# Define a function to process a single subject and extract PSD features
def process_subject(raw_data, sfreq):
    # Apply bandpass filtering and artifact removal here as per previous preprocessing steps
    set_montage(raw_data)
    raw_filtered = bandpass_filter(raw_data)  # Assuming bandpass_filter function is defined
    raw_filtered = apply_ica(raw_filtered)
    epochs = segment_data(raw_filtered)  # Assuming segment_data function is defined to get (180, 32, 512)
    eeg_data = epochs[:, :, :512]  # Shape (180, 32, 512)

    # Compute PSD features for this subject
    psd_features = compute_psd(eeg_data, sfreq)  # Dictionary with PSD for each frequency band
    return psd_features

In [ ]:
# Loop over all subjects and store PSD features
def collect_psd_features(subject_files, sfreq):
    all_psd_features = {'delta': [], 'theta': [], 'alpha': [], 'beta': [], 'gamma': []}
    for subject_file in subject_files:
        # Load subject's data
        raw_data = mne.io.read_raw_bdf(subject_file, preload=True)
        raw_data.crop(tmax=180.)
        raw_data = raw_data.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status'])
        # Process each subject's data to extract PSD features
        psd_features = process_subject(raw_data, sfreq)
        print("Shape of psd_features[delta] in each sub: ",np.asarray(psd_features['delta']).shape)
        # Append features for each frequency band
        for band in all_psd_features.keys():
            all_psd_features[band].append(psd_features[band])  # Each entry: (180, 32)
    return all_psd_features


Each subject’s PSD feature for the δ band should have a shape of 
180
×
32
180×32 (180 time samples, 32 channels).

In [ ]:
# Collect PSD features for each group
sfreq = 512  # Sample frequency as given

For PD_on or PD_off

In [ ]:
psd_features_pd = collect_psd_features(subject_files_pd_on, sfreq)
# psd_features_pd = collect_psd_features(subject_files_pd_off, sfreq)

In [ ]:
np.asarray(psd_features_pd['delta']).shape

For HC

In [ ]:
psd_features_hc = collect_psd_features(subject_files_hc, sfreq)

In [ ]:
np.asarray(psd_features_hc['delta']).shape

In [ ]:
# Combine PSD features into a dataset for SVM classification
psd_features = {'delta': psd_features_pd['delta'] + psd_features_hc['delta'],
                'theta': psd_features_pd['theta'] + psd_features_hc['theta'],
                'alpha': psd_features_pd['alpha'] + psd_features_hc['alpha'],
                'beta': psd_features_pd['beta'] + psd_features_hc['beta'],
                'gamma': psd_features_pd['gamma'] + psd_features_hc['gamma']}

In [ ]:
np.asarray(psd_features['delta']).shape

In [ ]:
# Prepare dataset (example: delta band, label 0 for HC and 1 for PD_ON,OFF)
def prepare_dataset(psd_features, num_pd=15, num_hc=16, label_pd=1, label_hc=0):
    # Combine PSD features for PD_ON and HC
    data = []
    labels = []
    
    # Append PD subjects
    for i in range(num_pd):
        psd_pd_on_off = psd_features['delta'][i]  # Shape (180, 32)
        data.append(psd_pd_on_off)
        labels.extend([label_pd] * psd_pd_on_off.shape[0])  # Label each sample in this subject's data as PD_ON
    
    # Append HC subjects
    for i in range(num_hc):
        psd_hc = psd_features['delta'][i + num_pd]  # Assuming next in sequence
        data.append(psd_hc)
        labels.extend([label_hc] * psd_hc.shape[0])  # Label each sample in this subject's data as HC
    
    # Stack data into final dataset
    data = np.vstack(data)  # Resulting shape: (5580, 32)
    labels = np.array(labels)  # Resulting shape: (5580,)
    return data, labels

In [ ]:
# Prepare dataset (using only δ band for this example)
data, labels = prepare_dataset({'delta': psd_features['delta']})

In [ ]:
# Verify the data shape
print("Full dataset shape:", data.shape)  # Should be (5580, 32)
print("Label distribution:", np.unique(labels, return_counts=True))  # Should show counts for PD_ON and HC

In [ ]:
import pickle

# Save using pickle
with open('new_data/pd_on vs hc/delta.pkl', 'wb') as f:
    pickle.dump(
        {
            'data': data, 
            'labels': labels
        },
    f)